In [1]:
import picos as pic
import fun_dual as f
from itertools import product
from qiskit.quantum_info import pauli_basis,Pauli

# Lovasz SDP

In [2]:
n,d=4,3 # AME(4,3)

paulis = pauli_basis(n, True)

paulis=[p for p in paulis if not 0<f.wt(str(p),n)<d]  # remove all the paulis with less weight than d

E= {ind:p.to_label() for ind, p in enumerate(paulis)}

N=len(E)
all=range(N)

sdp = pic.Problem()
X = pic.SymmetricVariable("X", N) #variable

obj_fun=pic.sum([X[i,i] for i in range(N)])
sdp.set_objective('max', obj_fun) # objective function 

sdp.add_list_of_constraints(X[i,i]==X[i,0] for i in range(1,N)) 
sdp.add_constraint(X[0,0]==1) # lovasz constraints

sdp.add_list_of_constraints(X[i,j]==0 for i,j in product(all,all) if Pauli(E[i]).anticommutes(Pauli(E[j]))==True)
# sdp.add_list_of_constraints(X[i,j]==0 for i,j in product(all,all) if 1<f.wt(E[i],n)<d or 
#                             1<f.wt(E[j],n)<d or 
#                             1<f.wt(str(Pauli(E[i]).dot(Pauli(E[j]))),n)<d) #lovasz zeros

# sdp.add_list_of_constraints(X[i,j]==0 for i,j in product(all,all) if 
#                             1<f.wt(str(Pauli(E[i]).dot(Pauli(E[j]))),n)<d) #lovasz zeros

sdp.add_constraint(X>>0) 

sol = sdp.solve(solver='mosek', primals=True)

In [3]:
print(sol.problemStatus)
print(obj_fun.value,2**n) #comparison between maximum and upperbound 

feasible
14.415100286476706 16
